### 基礎設定

In [ ]:
import os
from dotenv import load_dotenv 
from langchain_openai import ChatOpenAI

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-4o-mini")

### LangChain 1.0 Agent 基礎使用

In [ ]:
from langchain.agents import create_agent
from langchain_community.tools.tavily_search import TavilySearchResults

web_search = TavilySearchResults(max_results=2)

agent = create_agent(
    model=llm,
    tools=[web_search],
    system_prompt="你是一名專業的智能助理，可以調用工具幫助用戶解決問題。"
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "請幫我查詢2024年諾貝爾物理學獎得主是誰？"}]}
)

In [ ]:
result['messages'][-1].content

### 基礎 Agent 流程解析

In [ ]:
web_search?

In [ ]:
web_search.invoke("截至目前為止，2025鳳凰颱風哪裡可以停班停課?")

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "截至目前為止，2025鳳凰颱風哪裡可以停班停課?"}]}
)
result

### 工具調用

In [ ]:
import os
from datetime import datetime
import requests
import json
from langchain.tools import tool

In [ ]:
@tool
def get_weather(lat, lon):
    """
    查詢即時天氣的函式。

    :param lat: 必要參數，浮點數 (float) 類型，
                用來表示要查詢天氣的具體緯度。

    :param lon: 必要參數，浮點數 (float) 類型，
                用來表示要查詢天氣的具體經度。

    :return: 來自 Open Metro API 的即時天氣查詢結果。
            (請求 URL: https://api.open-meteo.com/v1/forecast)

            回傳的物件是「解析後的 JSON 格式物件」，並以「字串」形式表示，
            其中包含了所有重要的天氣資訊。
    :rtype: str
    """

    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&hourly=temperature_2m"

    response = requests.get(url)
    
    data = response.json()
    return json.dumps(data)

In [ ]:
@tool
def write_file(content: str) -> str:
    """
    將指定內容寫入本地文件。

    :param content: 必要參數，字串 (str) 類型，
                    用來表示需要寫入文件的具體內容。

    :return: 寫入結果提示資訊。
    """
    try:
        # ✅ 始終先定義文件名（防止未綁定變量）
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"output_{timestamp}.txt"

        # 寫入文件
        with open(filename, "w", encoding="utf-8") as f:
            f.write(content)

        abs_path = os.path.abspath(filename)
        return f"✅ 已成功寫入本地文件：{abs_path}"

    except Exception as e:
        return f"❌ 文件寫入失敗：{str(e)}"

In [ ]:
agent = create_agent(
    model=llm,
    tools=[get_weather, write_file],
    system_prompt="你是一名多才多藝的智能助手，可以調用工具幫助用戶解決問題。"
)

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "请帮我查詢台北、東京、倫敦等地天氣，並且寫入本地文件。"}]}
)

In [ ]:
result

### 基礎 Agent 記憶管理

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

tools = [get_weather]

agent = create_agent(model=llm, 
                     tools=tools,
                     checkpointer=checkpointer)

In [ ]:
config = {
    "configurable": {
        "thread_id": "10200"  
    }
}

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "你好，我叫阿明！"}]},
    config
)
response

In [ ]:
latest = agent.get_state(config)
latest

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "請問我叫做什麼名字"}]},
    config
)
response

### 練習

- 調用其他 Agent

- 使用 Database 進行記憶管理
- 跨 Session 的記憶管理